In [10]:
import os
from session import session
import requests

headers = {"Cookie": "session=" + session}
url = "https://adventofcode.com/2023/day/1/input"
r = requests.get(url, headers=headers)
open("inputs23/1", "w").write(r.text)

21588

In [48]:
day1 = open("inputs23/1", "r").read()
# day1 = """1abc2
# pqr3stu8vwx
# a1b2c3d4e5f
# treb7uchet
# """
# total = 0
# for l in day1.split("\n"):
#     print(l)
#     if l:
#         first, last = None, None
#         for c in l:
#             if c.isnumeric():
#                 if first is None:
#                     first = c
#                 last = c
#         total += int(first + last)
# print(total)

day1 = open("inputs23/1", "r").read()



# day1 = """two1nine
# eightwothree
# abcone2threexyz
# xtwone3four
# 4nineeightseven2
# zoneight234
# 7pqrstsixteen
# """
# nums = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"]

1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet

142
